In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

# Note regarding this analysis:

**Welcome to my Notebook!**
**I'm just getting started in Data Analysis & working with Dataframes. Therefore my target is not to make a full analysis, but getting comfortable with Dataframes & data visualisation**

**I'm thankful for any helpful comments and input, so that I can improve my skill.**

# My imported packages

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
from matplotlib.pyplot import figure
import statistics

# Inspection of 1st Dataset (InjuryRecord)

In [ ]:
injuryrec = pd.read_csv('/kaggle/input/nfl-playing-surface-analytics/InjuryRecord.csv')
injuryrec.head()

Visualisation of the injured Body Parts

In [ ]:
plt.figure(figsize=(8,8))
injuries = sns.countplot(x = 'BodyPart', data = injuryrec)
plt.title('Injured Body Parts')
plt.show()

**The most injuries occur in Knees and Ankles. Heel injuries have the least part in the over all injuries**

How are the injuries distributed when we look at the Surface?

In [ ]:
# plot of FieldTyp per Indoor & Outdoor Game 
plt.figure(figsize=(15,5))
sns.countplot(x = 'BodyPart', data = injuryrec, hue = 'Surface')
plt.title('Injured Body Parts by Surfaces')
plt.xlabel('Injured Body Parts')
plt.ylabel('Number of Injuries')
plt.show()

There is no difference in the number of knee injuries on different surfaces. 
There are more ankle and toe injuries when played on synthetic surface.
General food injuries seem to occur more often on natural turf.

How long are players ,,out of the game" with those injuries?

In [ ]:
l = [injuryrec['DM_M1'].count()]  # list of the player unjured per time point
l.append(injuryrec['DM_M7'].value_counts()[1])
l.append(injuryrec['DM_M28'].value_counts()[1])
l.append(injuryrec['DM_M42'].value_counts()[1])


l2 = []
for i in range(0, len(l)-1):
        l2.append(l[i] - l[i+1])
l2.append(injuryrec['DM_M42'].value_counts()[1])
#-----------------------------------------------------for the graphic -----------------------------------------------------
l3 = ['1 day', '7 days', '28 days', '42 days'] # label
plt.figure(figsize=(10,5))
plt.title('How many players are unable to play for how long?') #title
plt.ylabel('Numbers of players')
plt.bar(l3, l2)
plt.show()

**----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------**
# 2nd Dataset (PlayList)

loading the dataset

In [ ]:
playlist = pd.read_csv('/kaggle/input/nfl-playing-surface-analytics/PlayList.csv')
playlist.head()

In [ ]:
plt.figure(figsize=(18,5))
sns.countplot(x = 'Temperature', data = playlist)
plt.xlabel('Temperature in °F')
plt.ylabel('Number of Games')
plt.show()

There is a large amount of ,,nonsese" values, saying there were -999°F (-572°C) in the stadium.
- Not sure how to proceed with this values. 
    - change them to zero is not possible because in history there were games where the temperature was below 0°F!
    - dropping them would mean huge data loss!
        
- a reasonable value would be -9°F but can't be sure that this is the true value and therefore would corrupt the data

**I left this problem up to now because I could not find a good solution**

# In what kind of Stadiums did the games take place?

When we look at the content of the 'StadiumType' column we see that there are a lot of redundant categories.
Some with spelling mistakes (Ourdoor) or variances of the same category ( Outdoor & Outdoors)
This has to be cleaned.
There are some strange categories like: Dome, Domed, Retractable Roof, Bowl and Cloudy which don't give much insight into the StadiumType or the status of Indoor or Outdoor.

In [ ]:
#stadium = playlist.groupby('StadiumType').count()
plt.figure(figsize=(10,10))
sns.countplot(y = 'StadiumType', data = playlist) # the plot is better visible when turned! 
plt.xlabel('Number of Games')
plt.ylabel('Types of Stadium')
plt.show()

* It's clear that the 'StadiumType' category has to be cleaned thoroughly
* there are redundant categories and misspelling.... (Outddors, Oudoor) which have to be corrected & combined
* what does ,,Open" mean? (Open Dome maybe?)

**most of the games were played outdoor**

I simplified the categories of the StadiumType Column by exchanging the Data

In [ ]:
# cleaning the StadiumType column:
d = {'Oudoor': 'Outdoor', 'Outdoors': 'Outdoor', 'Ourdoor': 'Outdoor', 'Outdor': 'Outdoor', 'Outside': 'Outdoor', 'Indoors' : 'Indoor', 'Outddors' : 'Outdoor', \
     'Domed, closed': 'Closed Dome', 'Domed': 'Dome', 'Domed, Open':'Open Dome', 'Dome, closed' : 'Closed Dome', 'Retr. Roof-Closed' : 'Retr. Roof Closed',\
     'Retr. Roof - Closed': 'Retr. Roof Closed', 'Outdoor Retr Roof-Open': 'Retr. Roof-Open', 'Indoor, Open Roof' : 'Retr. Roof-Open', 'Indoor, Roof Closed': 'Retr. Roof Closed',\
     'Retr. Roof - Open': 'Retr. Roof-Open','Heinz Field' : 'Outdoor'}
playlist_mod = playlist.replace(d)

playlist_mod['StadiumType'].unique() #checking for unique categories

In [ ]:
plt.figure(figsize=(10,10))
sns.countplot(y = 'StadiumType', data = playlist_mod) # the plot is better visible when turned! 
plt.xlabel('Number of Games')
plt.ylabel('Types of Stadium')
plt.show()

I cleaned the category as careful as possible. 
In my opinion the two categories 'Bowl' & 'Cloudy' could be droped when there is a focus on StadiumType and FieldType because they do not contribute to the analysis here.

In the following I combined all the StadiumTypes to two categories: ,,Indoor" & "Outdoor":

In [ ]:
d2 = {'Open' : 'Outdoor', 'Closed Dome' : 'Indoor', 'Dome' : 'Indoor', 'Retr. Roof Closed' : 'Indoor', 'Retr. Roof-Open' : 'Outdoor', 'Retractable Roof' : 'Indoor', 'Open Dome' : 'Outdoor', 'Domed, open' : 'Outdoor' }
playlist_mod2 = playlist_mod.replace(d2)

And I droped ,,Bowl" & "Cloudy" from the StadiumType column

In [ ]:
# drop lines with a certain content from a column
indexNames = playlist_mod2[playlist_mod2['StadiumType'] == 'Bowl'].index
playlist_mod2.drop(indexNames, inplace = True)
indexNames = playlist_mod2[playlist_mod2['StadiumType'] == 'Cloudy'].index
playlist_mod2.drop(indexNames, inplace = True)
playlist_mod2['StadiumType'].unique()

In [ ]:
plt.figure(figsize=(8,8))
sns.countplot(x = 'StadiumType', data = playlist_mod2) # the plot is better visible when turned! 
plt.ylabel('Number of Games')
plt.xlabel('Types of Stadium')
plt.show()

**There are in general more games played outdoor than indoor**

In [ ]:
# plot of FieldTyp per Indoor & Outdoor Game 
plt.figure(figsize=(20,8))
sns.countplot(x = 'StadiumType', data = playlist_mod2, hue = 'FieldType')
plt.title('Distribution of Synthetic and Natural turf in games played indoor or outdoor')
plt.xlabel('Field types')
plt.ylabel('Number of Games')
plt.show()

** Outdoor games are played more on natural turf that synthetic one. Indoor games however are played on a synthetic field**

# How is generally the weather during the recorded games?

The weather and the surface together could have an impact on the injuries. For example could rain make the natural surface more slippery which could lead to more injuries.

In [ ]:
plt.figure(figsize=(15,15))
sns.countplot(y = 'Weather', data = playlist) # the plot is better visible when turned! 
plt.title('Recorded weather categories ')
plt.xlabel('Number of Games')
plt.ylabel('Weather conditions')
plt.show()

* same which holds true for the 'Weather' Category
    * spelling mistakes, redundant categories (Clear and Sunny & Clear and sunny)
    * the same ,,cleaning of categories" should be done, but could lead do corruption of the data.
    
** during most of the games the weather was cloudy **


Let's have a look at the players positions:

In [ ]:
plt.figure(figsize=(20,5))
sns.countplot(x = 'RosterPosition', data = playlist) # the plot is better visible when turned! 
plt.title('Player Positions')
plt.xlabel('Positions')
plt.ylabel('Number of records')
plt.show()

# Merging Dataframes

In [ ]:
injuryrec.head()

In [ ]:
playlist_mod.head()

In [ ]:
inplay = pd.merge(injuryrec, playlist_mod, on = 'PlayKey')


In [ ]:
inplay.info()

In [ ]:
inplay.tail()

**After merging the Dataframes we can look more deeply into the conections of Injuries and the other categories**

In [ ]:
plt.figure(figsize=(15,5))
sns.countplot(x = 'PlayType', data = inplay, hue = 'BodyPart') 
plt.title('Injuries per Play Type')
plt.xlabel('Play Type')
plt.ylabel('Number of injuries')
plt.show()

The most knee injuries occur during Pass plays and the most Ankle injuries during Passes & Rushes. But in general Knee and Ankle injuries are occuring throughout all the plays exept for 'Punt Not Returned'. Plays with high amount of movement like passes and rushes seem to cause more injuries than static plays as 'Punt Not Returned'.

In [ ]:
plt.figure(figsize=(15,5))
sns.countplot(x = 'PositionGroup', data = inplay, hue = 'BodyPart') 
plt.title('Injuries per Position Group')
plt.xlabel('Position')
plt.ylabel('Number of injuries')
plt.show()

Linebackers, defensive backs, wide reciever and running backs have high values of knee and ankle injuries. In general the offensive line shows more ankle injuries and the defensive line more Knee injuries.

But we can see that quaterbacks are not mentioned in this plot. So they seem to have few injuries of the lower body region.

We can go more into detail of the injuries of the different players position:

In [ ]:
plt.figure(figsize=(15,8))
sns.countplot(x = 'Position', data = inplay, hue = 'BodyPart') 
plt.title('Injuries per Position Group')
plt.xlabel('Position')
plt.ylabel('Number of injuries')
plt.show()

Outside Linebackers (OLB) are part of the defense line and as already seen in the previous plot they show twice as much knee injuries than ankle injuries. This might be due to their static position in defending pass play or running play. Same can be seen for an other position of the defensive line like Inside linebackers (ILB).

Wide recievers are running down the field as part of the offensive team and show knee injuries as well as ankle injuries in nearly the same amounts. They are fast sprinters so who accalerate fastly and stopp abruptly. 

In general OLB & WR ,,get hurt" according to this data. 

In [ ]:
plt.figure(figsize=(15,5))
sns.countplot(x = 'PositionGroup', data = inplay, hue = 'Surface') 
plt.title('Injuries per Position Group')
plt.xlabel('Position Group')
plt.ylabel('Number of injuries')
plt.figure(figsize=(15,5))
sns.countplot(x = 'Position', data = inplay, hue = 'Surface') 
plt.title('Injuries per Position')
plt.xlabel('Position')
plt.ylabel('Number of injuries')
plt.show()

In general D-line & O-line show the majority of injuries on a natural surface (upper plot). 
Tight ends however injur themself equally no matter what the surface is. :D 
For OLB the most injuries occur on natural turf whereas for ther wide recievers most of the injuies occur on a synthetic surface. For corner backs (CB) the majority of injuries occur on synthetic surfaces.

When we look under which weather conditions this injuries occur:  

**NOTE: I did not clear redundant categories of the weather column!**

In [ ]:
plt.figure(figsize=(15,6))
sns.countplot(x = 'BodyPart', data = inplay, hue = 'Weather') 
plt.title('Injuried Body Part')
plt.xlabel('Body Part')
plt.ylabel('Number of injuries')
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.) # putting the legend outside of the figure
plt.figure(figsize=(15,10))
sns.countplot(x = 'BodyPart', data = inplay, hue = 'Temperature') 
plt.title('Injuried Body Part by Temperature')
plt.xlabel('Body Part')
plt.ylabel('Number of injuries')
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.) # putting the legend outside of the figure
plt.figure(figsize=(15,10))
sns.countplot(x = 'Surface', data = inplay, hue = 'Temperature') 
plt.title('Surface by Temperature')
plt.xlabel('Surface')
plt.ylabel('Number of injuries')
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.) # putting the legend outside of the figure
plt.show()

Injuries occur on synthetic surface throughout the whole recorded temperature range. But mostly around 65°F. On natural turf however there are more injuries at higher temperatures. The Knee injuries occur mostly at temperatures of 65°F and higher and during cloudy, sunny or partially sunny weather, whereas ankle injuries are occuring during all kind of temperatures but as well during cloudy, sunny or partially sunny weather. 

**-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
**
# 3rd Dataset (player)

In [ ]:
player = pd.read_csv('/kaggle/input/nfl-playing-surface-analytics/PlayerTrackData.csv')
player.head()

In [ ]:
total = pd.merge(inplay, player, on = 'PlayKey')
total.head()

In [ ]:
fig = px.scatter_matrix(total,
    dimensions=['Surface', 's'],
    color="BodyPart")
fig.show()